In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplcyberpunk

plt.style.use("cyberpunk")
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from tqdm import tqdm
import torchutils as tu
from sklearn.metrics.pairwise import cosine_similarity
torch.manual_seed(666)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
df = pd.read_csv("/home/artemiy/recsys_project/data/dataset_itog.csv")
df.head()

,url,poster,title,description,rating,year,genres,cast,country,age
0,https://kino.mail.ru/series_915957_potapov_i_l...,https://resizer.mail.ru/p/a4bff38f-c8db-57ac-9...,Гадалка,"Люся видит во сне мертвое тело своей сестры, б...",6.4,2019,"Драма, Детектив, Мистика","Михаил Пореченков, Екатерина Олькина, Яна Край...",Россия,16 +
1,https://kino.mail.ru/series_773009_ohotnik/,https://resizer.mail.ru/p/dccc9300-db25-55ad-b...,Охотник,"Средневековый охотник — Владимир, сын Казатула...",7.4,2006,"Фантастика, Фэнтези, Приключения","Евгений Цыганов, Елена Панова, Сергей Никоненк...",Россия,16 +
2,https://kino.mail.ru/series_925738_velikaya/,https://resizer.mail.ru/p/2bebfd49-f476-5000-9...,Великая,Сериал о молодости императрицы Екатерины II. М...,4.8,2020,"Драма, Комедия","Эль Фаннинг, Николас Холт, Фиби Фокс, Адам Год...",США,18 +
3,https://kino.mail.ru/series_784898_fantastiche...,https://resizer.mail.ru/p/68617b86-fd8b-5e43-b...,Фантастическая девушка,Главная героиня сериала — Эви — фантастическая...,6.5,1987,"Фантастика, Комедия, Семейный","Донна Пескоу, Морин Флэннигэн, Берт Рейнолдс, ...",США,0 +
4,https://kino.mail.ru/series_785356_shpionka/,https://resizer.mail.ru/p/733fedb7-047d-58d5-9...,Шпионка,"Переезды по всему миру, борьба с «плохими парн...",7.1,2001,"Драма, Боевик, Криминал, Триллер, Детектив","Дженнифер Гарнер, Майкл Вартан, Рон Рифкин, Ви...",США,16 +


In [6]:
# pip install transformers sentencepiece
# import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
model.cuda()  # uncomment it if you have a GPU


def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()


print(embed_bert_cls("привет мир", model, tokenizer).shape)
# (312,)

/home/artemiy/miniforge-pypy3/envs/recsys/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(312,)


In [8]:
df.columns

Index(['url', 'poster', 'title', 'description', 'rating', 'year', 'genres',
       'cast', 'country', 'age'],
      dtype='object')

In [9]:
# Создание эмбеддингов для описаний сериалов
df["description_embedding"] = df["description"].apply(
    lambda x: embed_bert_cls(x, model, tokenizer)
)

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [11]:
df['description']

0        Люся видит во сне мертвое тело своей сестры, б...
1        Средневековый охотник — Владимир, сын Казатула...
2        Сериал о молодости императрицы Екатерины II. М...
3        Главная героиня сериала — Эви — фантастическая...
4        Переезды по всему миру, борьба с «плохими парн...
                               ...                        
15745    «Дикие животные: Завоевание мегаполиса» — это ...
15746    Харизматичный, но непредсказуемый наркодилер Т...
15747    Специальный документальный проект в рамках шоу...
15748    Нине чуть за тридцать и у нее расстройство аут...
15749    В течение пяти лет полиция пыталась поймать же...
Name: description, Length: 15750, dtype: object

In [ ]:
def find__shows(query, df, model, tokenizer, top_k=5):
    # Создание эмбеддинга для пользовательского запроса
    query_embedding = embed_bert_cls(query, model, tokenizer).reshape(1, -1)

    # Получение эмбеддингов описаний сериалов
    embeddings = np.stack(df["description_embedding"].values)

    # Вычисление косинусного сходства
    similarities = cosine_similarity(query_embedding, embeddings).flatten()

    # Получение топ-K самых похожих сериалов
    top_k_indices = similarities.argsort()[-top_k:][::-1]
    return df.iloc[top_k_indices]